In [1]:
from yapss import Problem
import numpy as np
import matplotlib.pyplot as plt
import casadi as cd

from helpers import inv_4x4, matmul_4d

In [2]:
### Acrobot parameters (taken from Spong 1995)
# Masses (kg)
m1 = 1.
m2 = 1.
# Link lengths (m)
l1 = 1.
l2 = 2.
# Centers of masses (m)
lc1 = 0.5
lc2 = 1.
# Moments of inertia
I1 = 0.083
I2 = 0.33
# Gravity
g = 9.8

In [3]:
### Problem parameters
t0 = 0
tf = 10
R = 10
x0 = [0, 0, 0, 0]

# Acrobot state: [theta1, theta2, theta1_dot, theta2_dot]
# input: [torque] (at elbow joint only)
problem = Problem(name="acrobot", nx=[4], nu=[1], nq=[1])

In [9]:
### Acrobot dynamics & objective function

def objective(arg):
    arg.objective = arg.phase[0].integral[0]

def continuous(arg):
    theta1, theta2, theta1_dot, theta2_dot = arg.phase[0].state
    u, = arg.phase[0].control  # elbow torque

    # M(q)*q_ddot + C(q, q_dot)*q_dot = tau_g(q) + Bu
    # q = [theta1, theta2]^T

    # M, mass matrix
    # M11 = I1 + I2 + m2*l1**2 + 2*m2*l1*lc2*cd.cos(cd.SX(theta2))
    # M12 = I2 + m2*l1*lc2*cd.cos(theta2)
    # M_val = [[M11, M12], [M12, I2]]
    # M = cd.SX(M_val)
    M = np.array([[I1 + I2 + m2*l1**2 + 2*m2*l1*lc2*np.cos(theta2), I2 + m2*l1*lc2*np.cos(theta2)],
                  [I2 + m2*l1*lc2*np.cos(theta2), cd.SX(I2)]])

    # C, Coriolis terms
    C = np.array([[-2*m2*l1*lc2*np.sin(theta2)*theta2_dot, -m2*l1*lc2*np.sin(theta2)*theta2_dot],
               [m2*l1*lc2*np.sin(theta2)*theta1_dot, 0]])

    # tau_g, gravity terms
    tau_g = np.array([-m1*g*lc1*np.sin(theta1) - m2*g*(l1*np.sin(theta1) + lc2*np.sin(theta1 + theta2)),
                   -m2*g*lc2*np.sin(theta1 + theta2)])

    # Combine to make the big matrix, call it M_block
    # NOTE I'm doing it this way because numpy tries to invert block matrices as their individual blocks
    # So even when the overall matrix is invertible, it says it's singular if C is singular
    M_block = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [C[0,0], C[0,1], M[0,0], M[0,1]],
                     [C[1,0], C[1,1], M[1,0], M[1,1]]])

    # arg.phase[0].dynamics[:] = cd.mtimes(cd.inv(M_block), cd.SX(np.array([theta1_dot, theta2_dot, tau_g[0], tau_g[1] + u])))
    arg.phase[0].dynamics[:] = np.matmul(M_block, np.array([theta1_dot, theta2_dot, tau_g[0], tau_g[1] + u]))
    # arg.phase[0].integrand[0] = R * u**2 # Cost only on input "effort"
    arg.phase[0].integrand[0] = 0.  # TODO only for testing

functions = problem.functions
functions.objective = objective
functions.continuous = continuous

In [ ]:
### TESTING
theta1 = np.pi
theta2 = 0.
theta1_dot = 0.
theta2_dot = 0.
u = 0.4

# M, mass matrix
M = np.array([[I1 + I2 + m2*l1**2 + 2*m2*l1*lc2*np.cos(theta2), I2 + m2*l1*lc2*np.cos(theta2)],
                [I2 + m2*l1*lc2*np.cos(theta2), I2]])

# C, Coriolis terms
C = np.array([[-2*m2*l1*lc2*np.sin(theta2)*theta2_dot, -m2*l1*lc2*np.sin(theta2)*theta2_dot],
              [m2*l1*lc2*np.sin(theta2)*theta1_dot, 0]])

# tau_g, gravity terms
tau_g = np.array([-m1*g*lc1*np.sin(theta1) - m2*g*(l1*np.sin(theta1) + lc2*np.sin(theta1 + theta2)),
                  -m2*g*lc2*np.sin(theta1 + theta2)])

# Combine to make the big matrix, call it M_block
M_block = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [C[0,0], C[0,1], M[0,0], M[0,1]],
                    [C[1,0], C[1,1], M[1,0], M[1,1]]])

dynamics = np.linalg.inv(M_block) @ np.array([theta1_dot, theta2_dot, tau_g[0], tau_g[1] + u])

In [5]:
### Set bounds
bounds = problem.bounds.phase[0]

# Initial conditions
bounds.initial_time.lower = bounds.initial_time.upper = t0
bounds.initial_state.lower = bounds.initial_state.upper = x0

# Final time
bounds.final_time.lower = bounds.final_time.upper = tf

# Control bounds
# TODO no control right now
bounds.control.lower[:] = bounds.control.upper[:] = (0,)

In [6]:
### Initial guess
phase = problem.guess.phase[0]
phase.time = (t0, tf)
phase.state = ((0, 0), (0, 0), (0, 0), (0, 0))
phase.control = ((0, 0),)

In [7]:
### Mesh and solver settings
m, n = 10, 10
problem.mesh.phase[0].collocation_points = m * (n,)
problem.mesh.phase[0].mesh_fraction = m * (1.0 / m,)

problem.derivatives.order = "second"
problem.derivatives.method = "auto"
problem.spectral_method = "lgl"
problem.ipopt_options.tol = 1e-10

In [10]:
### Solve
output = problem.solve()

TypeError: exceptions must derive from BaseException

In [ ]:
# extract information from solution
time = output.solution.phase[0].time
time_u = output.solution.phase[0].time_u
q1, q2, q1_dot, q2_dot = output.solution.phase[0].state
(torque,) = output.solution.phase[0].control
phase = output.solution.phase[0]

# initial and final time
t0 = output.solution.phase[0].t0
tf = output.solution.phase[0].tf

In [ ]:
plt.figure()
plt.plot(time, q2_dot)
plt.grid()